<a href="https://colab.research.google.com/github/Swag-With-Jal/Swag-With-Jal/blob/main/movie_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [ ]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
movies = movies.merge(credits, left_on='title', right_on='title')

In [ ]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['genres']

,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4804,"[Action, Crime, Thriller]"
4805,"[Comedy, Romance]"
4806,"[Comedy, Drama, Romance, TV Movie]"
4807,[]


In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies['keywords']

,keywords
0,"[culture clash, future, space war, space colon..."
1,"[ocean, drug abuse, exotic island, east india ..."
2,"[spy, based on novel, secret agent, sequel, mi..."
3,"[dc comics, crime fighter, terrorist, secret i..."
4,"[based on novel, mars, medallion, space travel..."
...,...
4804,"[united states–mexico barrier, legs, arms, pap..."
4805,[]
4806,"[date, love at first sight, narration, investi..."
4807,[]


In [ ]:
movies['cast'] = movies['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)[:3]])  # Only top 3 actors

In [ ]:
movies['crew'] = movies['crew'].apply(lambda x: [i['name'] for i in ast.literal_eval(x) if i['job'] == 'Director'])

In [ ]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags']

,tags
0,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,"[Action, Adventure, Crime, spy, based on novel..."
3,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,"[Action, Adventure, Science Fiction, based on ..."
...,...
4804,"[Action, Crime, Thriller, united states–mexico..."
4805,"[Comedy, Romance, Edward Burns, Kerry Bishé, M..."
4806,"[Comedy, Drama, Romance, TV Movie, date, love ..."
4807,"[Daniel Henney, Eliza Coupe, Bill Paxton, Dani..."


In [ ]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))
movies['tags']

,tags
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...
...,...
4804,Action Crime Thriller united states–mexico bar...
4805,Comedy Romance Edward Burns Kerry Bishé Marsha...
4806,Comedy Drama Romance TV Movie date love at fir...
4807,Daniel Henney Eliza Coupe Bill Paxton Daniel Hsia


In [ ]:
movies = movies[['movie_id', 'title', 'overview', 'tags']]
movies['tags'] = movies['tags'].apply(lambda x: x.lower())
movies.head()

<ipython-input-13-392060511f81>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: x.lower())


,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,action adventure crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,action crime drama thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",action adventure science fiction based on nove...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['tags'])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movies[movies['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [ ]:
print(get_recommendations('The Dark Knight Rises'))


65               The Dark Knight
119                Batman Begins
1360                      Batman
210               Batman & Robin
428               Batman Returns
1361                      Batman
1197                The Prestige
303                     Catwoman
4644    Amidst the Devil's Wings
72                 Suicide Squad
Name: title, dtype: object


In [ ]:
import pickle
with open('movie_data.pkl', 'wb') as file:
    pickle.dump((movies, cosine_sim), file)

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import requests
import pickle

# Load the processed data and similarity matrix
with open('movie_data.pkl', 'rb') as file:
    movies, cosine_sim = pickle.load(file)

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movies[movies['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies[['title', 'movie_id']].iloc[movie_indices]

In [ ]:
def fetch_poster(movie_id):
    api_key = 'b10dd61d5b001cf1b45b68cd796e3ca8'  # Replace with your TMDB API key
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
    response = requests.get(url)
    data = response.json()
    poster_path = data['poster_path']
    full_path = f"https://image.tmdb.org/t/p/w500{poster_path}"
    return full_path


In [ ]:
# Streamlit UI
st.title("Movie Recommendation System")

selected_movie = st.selectbox("Select a movie:", movies['title'].values)

if st.button('Recommend'):
    recommendations = get_recommendations(selected_movie)
    st.write("Top 10 recommended movies:")

2025-02-11 14:37:52.624 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:52.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
  # Create a 2x5 grid layout
  for i in range(0, 10, 5):  # Loop over rows (2 rows, 5 movies each)
      cols = st.columns(5)  # Create 5 columns for each row
      for col, j in zip(cols, range(i, i+5)):
          if j < len(recommendations):
              movie_title = recommendations.iloc[j]['title']
              movie_id = recommendations.iloc[j]['movie_id']
              poster_url = fetch_poster(movie_id)
              with col:
                  st.image(poster_url, width=130)
                  st.write(movie_title)

2025-02-11 14:37:54.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:54.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:54.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:54.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:54.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 14:37:54.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


NameError: name 'recommendations' is not defined